Part a) Ordinary Least Square (OLS) for the Runge function

In [2]:
#Starting by generating point of x in the interval [-1,1]

import numpy as np 

#choosing 100 datapoints between 1 and -1 
n = 100
x = np.linspace (-1,1, n)

#From the Runge function: for every x, compute the correspponding y
y = 1/(1 + 25 * x**2)

#Adding noise (Gaussian) with mean 0 and variance 1: N(0,1)
noise = np.random.normal(0,1,n)
y_noisy = y + 0.1 * noise